# Proyecto 1 - Laboratorio de Aprendizaje Estadístico

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import statsmodels.api as sm

Teoria:

- Regresion lineal
- Regresion polinomial
- Ridge
- Lasso
- p values
- elastic net

### Descarga de Datos

In [2]:
df = pd.read_csv("Hitters.csv")
df.dropna()
df = df[df['Salary'].notna()] 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, 1 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      263 non-null    int64  
 1   Hits       263 non-null    int64  
 2   HmRun      263 non-null    int64  
 3   Runs       263 non-null    int64  
 4   RBI        263 non-null    int64  
 5   Walks      263 non-null    int64  
 6   Years      263 non-null    int64  
 7   CAtBat     263 non-null    int64  
 8   CHits      263 non-null    int64  
 9   CHmRun     263 non-null    int64  
 10  CRuns      263 non-null    int64  
 11  CRBI       263 non-null    int64  
 12  CWalks     263 non-null    int64  
 13  League     263 non-null    object 
 14  Division   263 non-null    object 
 15  PutOuts    263 non-null    int64  
 16  Assists    263 non-null    int64  
 17  Errors     263 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  263 non-null    object 
dtypes: float64(1), 

In [3]:
data = pd.get_dummies(df, columns = ['League', 'Division', 'NewLeague'])

### Funciones

In [4]:
def datos(data):


    x = data.drop(columns = ['Salary'])
    y = data['Salary']
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state=42)
    scaler = StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
    x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)
    
    return x_train_scaled, x_test_scaled, y_train, y_test


def var_polinomiales(data, degree):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    pols = poly.fit_transform(data)
    feature_names = poly.get_feature_names_out(data.columns)
    data_poly = pd.DataFrame(pols, columns=feature_names, index=data.index)
    return data_poly


def datos_polinomiales(data, degree):
    x_train_scaled, x_test_scaled, y_train, y_test = datos(var_polinomiales(data, degree))
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    x_train_poly = poly.fit_transform(x_train_scaled)
    x_test_poly = poly.transform(x_test_scaled)
    
    feature_names = poly.get_feature_names_out(x_train_scaled.columns)
    
    x_train_poly = pd.DataFrame(x_train_poly, columns=feature_names, index=x_train_scaled.index)
    x_test_poly = pd.DataFrame(x_test_poly, columns=feature_names, index=x_test_scaled.index)
    
    return x_train_poly, x_test_poly, y_train, y_test


def dataframe_polinomial(degree):
    resultados = sin_penalizacion(data)[2]
    tabla = resultados.summary2().tables[1]
    tabla_filtrada = tabla[tabla['P>|t|'] > 0.2]  # no tomamos mayores a 0.05, sino los 8 valores de p-value mayores
    
    variables = tabla_filtrada.index.tolist()
    data_variables = data[variables]
    
    variables_poly = var_polinomiales(data_variables, degree=degree)
    
    data_nueva = data
    data_nueva = data.drop(columns=variables)
    
    data_nueva_variables = pd.concat([data_nueva, variables_poly], axis=1)
    return data_nueva_variables


def sin_penalizacion(data):


    x_train_scaled, x_test_scaled, y_train, y_test = datos(data)
    
    ols = sm.OLS(y_train, sm.add_constant(x_train_scaled))
    results = ols.fit()
    
    y_pred = results.predict(sm.add_constant(x_test_scaled))
    r2s = r2_score(y_test, y_pred)
    r2s = pd.DataFrame({'R2_test': [r2s]})
    res = results.summary()
    return res, r2s, results


def ridge(data, alpha):

    x_train_scaled, x_test_scaled, y_train, y_test = datos(data) 
    ridge = Ridge(alpha=alpha)
    ridge.fit(x_train_scaled, y_train)
    y_pred_train = ridge.predict(x_train_scaled)
    r2_train = r2_score(y_train, y_pred_train)
    y_pred = ridge.predict(x_test_scaled)
    r2 = r2_score(y_test, y_pred)

    res = pd.DataFrame({'Variables': x_train_scaled.columns, 'Coeficientes': ridge.coef_})
    r2s = pd.DataFrame({'R2_train': [r2_train], 'R2_test': [r2]})
    return res, r2s

def lasso(data, alpha):


    x_train_scaled, x_test_scaled, y_train, y_test = datos(data)
    lasso = Lasso(alpha = alpha, max_iter = 10_000)
    lasso.fit(x_train_scaled, y_train)
    y_pred_train = lasso.predict(x_train_scaled)
    r2_train = r2_score(y_train, y_pred_train)
    y_pred = lasso.predict(x_test_scaled)
    r2 = r2_score(y_test, y_pred)

    res = pd.DataFrame({'Variables': x_train_scaled.columns, 'Coeficientes': lasso.coef_})
    r2s = pd.DataFrame({'R2_train': [r2_train], 'R2_test': [r2]})
    return res, r2s

def elastic_net(data, alpha, ratio):

    
    x_train_scaled, x_test_scaled, y_train, y_test = datos(data)
    
    elastic_net = ElasticNet(alpha=alpha, l1_ratio=ratio, max_iter=10_000)
    elastic_net.fit(x_train_scaled, y_train)
    y_pred_train = elastic_net.predict(x_train_scaled)
    r2_train = r2_score(y_train, y_pred_train)
    y_pred = elastic_net.predict(x_test_scaled)
    r2 = r2_score(y_test, y_pred)

    res = pd.DataFrame({'Variables': x_train_scaled.columns, 'Coeficientes': elastic_net.coef_})
    r2s = pd.DataFrame({'R2_train': [r2_train], 'R2_test': [r2]})
    return res, r2s



### Regresión lineal

In [5]:
sin_penalizacion(data)[0]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.557
Method:                 Least Squares   F-statistic:                     13.12
Date:                Fri, 26 Sep 2025   Prob (F-statistic):           4.36e-24
Time:                        07:40:02   Log-Likelihood:                -1292.1
No. Observations:                 184   AIC:                             2624.
Df Residuals:                     164   BIC:                             2688.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         535.8383     21.185     25.294      0.000     494.008     577.668
AtBat        -191.3439    104.702     -1.828      0.069    -398.082      15.394
Hits          297.1558    109.671      2.710      0.007      80.607     513.705
HmRun          84.9538     64.213      1.323      0.188     -41.837     211.745
Runs          -36.1422     79.797     -0.453      0.651    -193.704     121.419
RBI           -74.9421     73.741     -1.016      0.311    -220.546      70.662
Walks          81.8926     41.687      1.964      0.051      -0.420     164.205
Years          96.4996     67.973      1.420      0.158     -37.716     230.716
CAtBat       -911.7612    386.421     -2.359      0.019   -1674.764    -148.759
CHits         457.4959    535.362      0.855      0.394    -599.594    1514.586
CHmRun         37.7194    177.950      0.212      0.832    -313.648     389.087
CRuns         526.0352    289.992      1.814      0.072     -46.564    1098.634
CRBI          169.5277    288.405      0.588      0.557    -399.938     738.994
CWalks       -163.8521     95.719     -1.712      0.089    -352.853      25.149
PutOuts        87.8384     24.105      3.644      0.000      40.241     135.436
Assists        53.1173     33.528      1.584      0.115     -13.085     119.319
Errors        -13.8171     30.738     -0.450      0.654     -74.511      46.877
League_A      -35.5627     20.839     -1.707      0.090     -76.710       5.584
League_N       35.5627     20.839      1.707      0.090      -5.584      76.710
Division_E     22.0914     11.094      1.991      0.048       0.185      43.998
Division_W    -22.0914     11.094     -1.991      0.048     -43.998      -0.185
NewLeague_A    15.3823     20.484      0.751      0.454     -25.065      55.829
NewLeague_N   -15.3823     20.484     -0.751      0.454     -55.829      25.065
==============================================================================
Omnibus:                        8.366   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.015   Jarque-Bera (JB):               10.475
Skew:                           0.322   Prob(JB):                      0.00531
Kurtosis:                       3.976   Cond. No.                     4.48e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.57e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [6]:
sin_penalizacion(data)[1]

,R2_test
0,0.380623


In [7]:
ridge(data, 1)[0]

,Variables,Coeficientes
0,AtBat,-199.559388
1,Hits,289.887062
2,HmRun,73.273902
3,Runs,-17.321385
4,RBI,-56.166674
5,Walks,75.590000
6,Years,35.772276
7,CAtBat,-320.705418
8,CHits,140.723455
9,CHmRun,49.719390


In [8]:
ridge(data, 1)[1]

,R2_train,R2_test
0,0.594157,0.403631


In [9]:
lasso(data, 1)[0]

,Variables,Coeficientes
0,AtBat,-2.141244e+02
1,Hits,3.025200e+02
2,HmRun,6.447799e+01
3,Runs,-2.195379e+01
4,RBI,-4.302229e+01
5,Walks,7.246259e+01
6,Years,4.424827e+01
7,CAtBat,-3.928808e+02
8,CHits,2.425419e+01
9,CHmRun,0.000000e+00


In [10]:
lasso(data, 1)[1]

,R2_train,R2_test
0,0.5971,0.404124


In [11]:
elastic_net(data, 1, 0.5)[0]

,Variables,Coeficientes
0,AtBat,15.808969
1,Hits,51.345039
2,HmRun,22.786881
3,Runs,32.841385
4,RBI,16.239586
5,Walks,23.333458
6,Years,16.829502
7,CAtBat,24.185327
8,CHits,37.727043
9,CHmRun,27.412571


In [12]:
elastic_net(data, 1, 0.5)[1]

,R2_train,R2_test
0,0.529303,0.353081


### Regresión grado 2

In [13]:
data_cuadrada = dataframe_polinomial(2)
data_cubica = dataframe_polinomial(3)

In [14]:
sin_penalizacion(data_cuadrada)[0]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     18.45
Date:                Fri, 26 Sep 2025   Prob (F-statistic):           4.69e-40
Time:                        07:40:03   Log-Likelihood:                -1195.6
No. Observations:                 184   AIC:                             2485.
Df Residuals:                     137   BIC:                             2636.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     535.8383     13.718     39.060      0.000     508.711     562.965
AtBat                      56.8659     81.924      0.694      0.489    -105.133     218.865
Hits                      -52.0170     90.656     -0.574      0.567    -231.284     127.250
HmRun                    -154.5649     58.373     -2.648      0.009    -269.993     -39.136
Walks                      17.0232     31.119      0.547      0.585     -44.512      78.558
Years                       3.7296     51.115      0.073      0.942     -97.347     104.807
CAtBat                  -1701.8105    315.094     -5.401      0.000   -2324.887   -1078.734
CRuns                     -19.2089    220.724     -0.087      0.931    -455.676     417.258
CWalks                      1.1541     75.298      0.015      0.988    -147.742     150.050
PutOuts                    79.2295     17.786      4.454      0.000      44.058     114.401
Assists                     6.2955     26.097      0.241      0.810     -45.310      57.901
League_A                  -31.2194     14.719     -2.121      0.036     -60.325      -2.114
League_N                   31.2194     14.719      2.121      0.036       2.114      60.325
Division_E                  8.7520      7.646      1.145      0.254      -6.368      23.872
Division_W                 -8.7520      7.646     -1.145      0.254     -23.872       6.368
Runs                     -171.8894     94.636     -1.816      0.072    -359.026      15.247
RBI                       100.7834     92.482      1.090      0.278     -82.093     283.660
CHits                    1483.6403    348.455      4.258      0.000     794.595    2172.686
CHmRun                    420.4772    222.126      1.893      0.060     -18.763     859.717
CRBI                     -359.3656    417.603     -0.861      0.391   -1185.146     466.415
Errors                     27.8114     40.540      0.686      0.494     -52.355     107.977
NewLeague_A                -0.1302     11.262     -0.012      0.991     -22.400      22.140
NewLeague_N                 0.1302     11.262      0.012      0.991     -22.140      22.400
Runs^2                   -252.3406    164.514     -1.534      0.127    -577.655      72.974
Runs RBI                  790.2563    293.502      2.693      0.008     209.877    1370.636
Runs CHits               1463.2962    588.494      2.487      0.014     299.590    2627.002
Runs CHmRun              1348.0405    510.513      2.641      0.009     338.536    2357.545
Runs CRBI               -2432.6505   1038.505     -2.342      0.021   -4486.223    -379.078
Runs Errors               212.1435     88.772      2.390      0.018      36.603     387.684
Runs NewLeague_A          -41.8250     52.950     -0.790      0.431    -146.530      62.880
Runs NewLeague_N          -96.5824     51.581  

In [15]:
sin_penalizacion(data_cuadrada)[1]

,R2_test
0,0.407619


In [16]:
ridge(data_cuadrada, 1)[0]

,Variables,Coeficientes
0,AtBat,-69.018819
1,Hits,140.851465
2,HmRun,-84.542601
3,Walks,48.699308
4,Years,-77.229070
5,CAtBat,-414.988389
6,CRuns,309.497035
7,CWalks,-74.573385
8,PutOuts,81.712894
9,Assists,-19.554098


In [17]:
ridge(data_cuadrada, 1)[1]

,R2_train,R2_test
0,0.799778,0.475286


In [18]:
lasso(data_cuadrada, 1)[0]

,Variables,Coeficientes
0,AtBat,0.000000e+00
1,Hits,3.409458e+01
2,HmRun,-9.548441e+01
3,Walks,2.376546e+01
4,Years,-3.652049e+01
5,CAtBat,-1.174362e+03
6,CRuns,1.837732e+02
7,CWalks,-0.000000e+00
8,PutOuts,8.261202e+01
9,Assists,-5.360429e+00


In [19]:
lasso(data_cuadrada, 1)[1]

,R2_train,R2_test
0,0.825276,0.470247


In [20]:
elastic_net(data_cuadrada, 1, 0.5)[0]

,Variables,Coeficientes
0,AtBat,0.854778
1,Hits,29.025143
2,HmRun,0.000000
3,Walks,15.301269
4,Years,4.895961
5,CAtBat,14.887193
6,CRuns,32.643846
7,CWalks,19.960686
8,PutOuts,55.999154
9,Assists,-0.000000


In [21]:
elastic_net(data_cuadrada, 1, 0.5)[1]

,R2_train,R2_test
0,0.631365,0.428992


### Grado 3

In [22]:
sin_penalizacion(data_cubica)[0]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     14.93
Date:                Fri, 26 Sep 2025   Prob (F-statistic):           1.27e-22
Time:                        07:40:03   Log-Likelihood:                -1059.4
No. Observations:                 184   AIC:                             2367.
Df Residuals:                      60   BIC:                             2765.
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                            535.8383      9.890     54.177      0.000     516.054     555.622
AtBat                            158.7750    107.340      1.479      0.144     -55.936     373.486
Hits                            -197.9426    129.699     -1.526      0.132    -457.378      61.493
HmRun                             14.9406     78.373      0.191      0.849    -141.829     171.711
Walks                            -12.8007     44.540     -0.287      0.775    -101.894      76.293
Years                            -42.0179     72.506     -0.580      0.564    -187.052     103.017
CAtBat                         -2151.5475    597.148     -3.603      0.001   -3346.021    -957.074
CRuns                           -295.0201    331.463     -0.890      0.377    -958.046     368.005
CWalks                           136.1641    148.644      0.916      0.363    -161.168     433.496
PutOuts                           44.4233     21.467      2.069      0.043       1.482      87.364
Assists                            6.0830     26.762      0.227      0.821     -47.449      59.615
League_A                         -41.1692     16.992     -2.423      0.018     -75.159      -7.180
League_N                          41.1691     16.992      2.423      0.018       7.180      75.159
Division_E                        -2.0504      8.612     -0.238      0.813     -19.277      15.177
Division_W                         2.0504      8.612      0.238      0.813     -15.177      19.277
Runs                            -350.8290    224.975     -1.559      0.124    -800.847      99.189
RBI                              -84.9153    222.644     -0.381      0.704    -530.270     360.439
CHits                           1712.6048    879.221      1.948      0.056     -46.098    3471.308
CHmRun                           721.4778    581.132      1.242      0.219    -440.959    1883.914
CRBI                            -971.2756   1299.291     -0.748      0.458   -3570.246    1627.694
Errors                           178.5498     88.737      2.012      0.049       1.050     356.050
NewLeague_A                       -0.7275     16.191     -0.045      0.964     -33.114      31.659
NewLeague_N                        0.7275     16.191      0.045      0.964     -31.659      33.114
Runs^2                          -322.8954    842.482     -0.383      0.703   -2008.109    1362.319
Runs RBI                        1713.6610   1416.794      1.210      0.231   -1120.348    4547.670
Runs CHits                       293.0028   3409.458      0.086      0.932   -6526.929    7112.934
Runs CHmRun                     -896.6297   2242.963     -0.400      0.691   -5383.224    3589.965
Runs CRBI                        -55.6496   5303.221     -0.010      0.992   -1.07e+04    1.06e+04
Runs Errors   

In [23]:
sin_penalizacion(data_cubica)[1]

,R2_test
0,-1.95472


In [24]:
ridge(data_cubica, 1)[0]

,Variables,Coeficientes
0,AtBat,-17.262003
1,Hits,67.866510
2,HmRun,-69.656346
3,Walks,33.648582
4,Years,-49.775180
...,...,...
173,Errors NewLeague_N^2,30.153108
174,NewLeague_A^3,13.898360
175,NewLeague_A^2 NewLeague_N,0.000000
176,NewLeague_A NewLeague_N^2,0.000000


In [25]:
ridge(data_cubica, 1)[1]

,R2_train,R2_test
0,0.875765,0.412473


In [26]:
lasso(data_cubica, 1)[0]

,Variables,Coeficientes
0,AtBat,0.474711
1,Hits,23.626147
2,HmRun,-67.675944
3,Walks,22.656311
4,Years,-19.801481
...,...,...
173,Errors NewLeague_N^2,0.000000
174,NewLeague_A^3,0.129337
175,NewLeague_A^2 NewLeague_N,0.000000
176,NewLeague_A NewLeague_N^2,0.000000


In [27]:
lasso(data_cubica, 1)[1]

,R2_train,R2_test
0,0.860321,0.417089


In [28]:
elastic_net(data_cubica, 1, 0.5)[0]

,Variables,Coeficientes
0,AtBat,2.471654
1,Hits,22.359702
2,HmRun,-11.181716
3,Walks,11.313165
4,Years,2.805896
...,...,...
173,Errors NewLeague_N^2,-0.688813
174,NewLeague_A^3,-0.000000
175,NewLeague_A^2 NewLeague_N,0.000000
176,NewLeague_A NewLeague_N^2,0.000000


In [29]:
elastic_net(data_cubica, 1, 0.5)[1]

,R2_train,R2_test
0,0.72348,0.411073
